In [1]:
import os
from glob import glob
import copy 

In [2]:
#from dask import delayed
#from dask import compute
#from dask.distributed import Client, progress
#client = Client(threads_per_worker=4, n_workers=20)

In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from root_pandas import read_root 
from ROOT import TFile

Welcome to JupyROOT 6.16/00


/home/jcordero/CMS/SMP_ZGamma/envSMP/lib/python3.6/site-packages/root_numpy/__init__.py:46: RuntimeWarning: numpy 1.14.3 is currently installed but you installed root_numpy against numpy 1.9.3. Please consider reinstalling root_numpy for this numpy version.
  RuntimeWarning)


In [4]:
'''
from guppy import hpy
h = hpy()
print(h.heap())
''';

In [5]:
from Common.CommonHelper  import CommonHelper
from Config.ConfigManager import ConfigManager
from Samples.DataSample   import DataSample
from Samples.DataFile     import DataFile
from Samples.Event        import Event
from Samples.DataFile     import DataFile
from Samples.DataEras     import DataEras
from Samples.ConfigData   import ConfigData
from Cuts.Cuts            import Cuts
from Plotter.Histo        import Histo
from Plotter.HistoVar     import HistoVar
from Plotter.HistoSample  import HistoSample
from Plotter.HistoEras    import HistoEras

In [6]:
def getSamplesPath(path):
    return glob(os.path.join(path,"*[!.][!s][!h]"))

def getSampleFromPath(path):
    return path.split("/")[-1]

def getFileFromPath(path):
    return path.split("/")[-1]

def getFiles(path,sample):
    return glob(os.path.join(path,sample,"output*[!v_0]*"))

In [7]:
def isPathEmpty(path,samp):
    file = glob(os.path.join(path, getSampleFromPath(samp),"output*"))
    if file:
        return False
    else:
        return True

In [8]:
def FillLoop(Sample, HStack, Cut):    
    for file in Sample:
        for i,iEvent in enumerate(file):
            #if i == 1:break
            event = Event(iEvent)
            event = Event(event[Cut.Pass(event,Region=None)])
            if file.name == 'DYJets' and Cut.VetoDY is not None:
                event = Event(event[Cut._vetoDY(event)])
            for histo in HStack:
                #print(histo.name)
                histo.fill( event.value(histo.name),
                            event.value('genWeight')*event.value('eventWeight')*Sample.getSF())
                '''
                if histo.name == "eventWeight":
                    histo.fill( event.value(histo.name) )
                else:
                    histo.fill( event.value(histo.name),
                                event.value('eventWeight')*Sample.getSF())
                '''
    return HStack

In [11]:
Manager = ConfigManager()
datapath     = "/home/jcordero/CMS/data/"
histsavepath = "/home/jcordero/CMS/SMP_ZGamma/histo/"
figpath      = "/home/jcordero/CMS/SMP_ZGamma/fig/"
Manager.CreateProject(histsavepath)
Manager.CreateProject(figpath)


PhotonRadiation = None
OppositeCharge = True
#VetoDY = True
VetoDY = False #right veto settings
#VetoDY = None

Cut = Cuts(PhotonRadiation = PhotonRadiation, OppositeCharge = OppositeCharge, VetoDY = VetoDY)

selection = "mumug"
#selection = "elelg"

#chuncksize = 5000
chuncksize = 10000
#chuncksize = 1000

#eras = ['2016','2017','2018']
#eras = ['2017','2018']
eras = ['2016']
#eras = ['2017']
#eras = ['2018']

DEra = DataEras()
HEra = HistoEras(name = "ZGamma")
HUnmergeEra = HistoEras()
for era in eras:
    histsavepath = "/home/jcordero/CMS/SMP_ZGamma/histo/"
    figpath      = "/home/jcordero/CMS/SMP_ZGamma/fig/"
    dirstructure = os.path.join(era,selection,Manager.Date())
    
    print('Loading era {} for selection {}'.format(era,selection))
    path = datapath+"data_{}/{}/SMP_ZG/Files/{}/{}/".format(era,Manager.legacy(era),selection,Manager.latestDir(era,selection))
    histsavepath = os.path.join(histsavepath,dirstructure)
    figpath = os.path.join(figpath,dirstructure)
    Manager.CreateFigStructure(figpath)
    

    print('---Building data files')
    DSample = DataSample()
    for samp in getSamplesPath(path):
        if not isPathEmpty(path,samp) :
            DSample.append( DataFile(path, getSampleFromPath(samp), era, chuncksize) )
    DEra.append(DSample)
    
    Samples = DSample.getSamples()
    #Samples.pop(Samples.index('ZGToLLG'))
    #Samples.pop(Samples.index('DYJets'))

    #Samples = ['ZGToLLG','DYJets']
    #Samples = ['ZGToLLG']

    print('---Filling in histograms')
    HSample = HistoSample()
    for sample in Samples:
        print(sample)
        HVar = HistoVar()
        HSample.append(FillLoop(DSample[sample], HVar.initialize(), Cut), sample)
    
    HUnmergeSmaple = copy.deepcopy(HSample)
    HSample.merge(ConfigData.DoubleMuon(era),name ="DoubleMuon")
    HSample.merge(ConfigData.Diboson(era),name ="Diboson")
    
    HEra.append(HSample, name = era)
    HUnmergeEra.append(HUnmergeSmaple, name = era)


Loading era 2016 for selection mumug
---Building data files
---Filling in histograms
WWTo2L2Nu
DYJets
DoubleMuon_2016E
ZZTo4L
DoubleMuon_2016H


OSError: cannot read /home/jcordero/CMS/data/data_2016/legacy/SMP_ZG/Files/mumug/V2/DoubleMuon_2016H/output_DoubleMuon_2016H_v1_7.root

Error in <TFile::ReadBuffer>: error reading all requested bytes from file /home/jcordero/CMS/data/data_2016/legacy/SMP_ZG/Files/mumug/V2/DoubleMuon_2016H/output_DoubleMuon_2016H_v1_7.root, got 262 of 300
Error in <TFile::Init>: /home/jcordero/CMS/data/data_2016/legacy/SMP_ZG/Files/mumug/V2/DoubleMuon_2016H/output_DoubleMuon_2016H_v1_7.root failed to read the file type data.


In [ ]:
path

In [ ]:
ind  = ['DoubleMuon','WJets', 'TTTo2L2Nu','Diboson' , 'DYJets', 'ZGToLLG']

In [ ]:
for i,smp in enumerate(HSample):
    print(smp.name,smp.size())

In [ ]:
era ='2016'
figpath = "/home/jcordero/CMS/SMP_ZGamma/fig/"
dirstructure = os.path.join(era,selection,Manager.Date())
figpath = os.path.join(figpath,dirstructure)

variable = 'dileptonM'
fig, ax = HEra[era].plot( variable=variable,log=True,
                        ind = ind,
                        WithYield = True,)
                        #xranges = [50,200],)
                        #yranges=[0,12000])#,Debug=Tru
HEra[era].savefig(fig,os.path.join(figpath,variable))

In [ ]:
eraind = {
        '2016':0,
        '2017':1,
        '2018':2,
        }
era = '2018'
path = "/home/jcordero/CMS/data/data_{}/{}/SMP_ZG/Files/{}/VDask/".format(era,Manager.legacy(era),selection)
histsavepath += os.path.join(era,selection,Manager.Date())

figpath = "/home/jcordero/CMS/SMP_ZGamma/fig/"
dirstructure = os.path.join(era,selection,Manager.Date())
figpath = os.path.join(figpath,dirstructure)

In [ ]:
HEra[eraind[era]].savefig(fig,os.path.join(figpath,variable))

In [ ]:
histsavepath = "/home/jcordero/CMS/SMP_ZGamma/histo/"
dirstructure = os.path.join(era,selection,Manager.Date())
histsavepath = os.path.join(histsavepath,dirstructure)

HEra[eraind[era]].savehists(path = histsavepath)

In [ ]:
HEra[0].name

In [ ]:
#HEra[eraind[era]].savefigs(figpath)
#HEra[0].savefigs(figpath, ind=ind)

#HEra[0].savefigs(figpath, ind=ind)
era ='2018'
figpath = "/home/jcordero/CMS/SMP_ZGamma/fig/"
dirstructure = os.path.join(era,selection,Manager.Date())
figpath = os.path.join(figpath,dirstructure)
HEra[2].savefigs(figpath, ind=ind)

In [ ]:
x = np.arange(0,10)
xc = [x for _ in range(3)]
wei  = [10*x for _ in range(3)]
mchist = plt.hist(xc,weights=wei,stacked=True)
ax = plt.gca()
ax.set_xticklabels([])
for i in dir(ax):
    if 'tick' in i:
        print(i)

In [ ]:
ax.set_xticks()

In [ ]:
len(mchist[1])

In [ ]:
mchist[0][-1]

In [ ]:
figpath = "/home/jcordero/CMS/SMP_ZGamma/fig/"
dirstructure = os.path.join(era,selection,Manager.Date())
figpath = os.path.join(figpath,dirstructure)


In [ ]:
Manager.CreateFigStructure(figpath)

In [ ]:
x = np.arange(0,10)
plt.plot(x)

In [ ]:
from ROOT import TFile
#from Cuts.Cuts import Cuts
#from Samples.Event import Event
from root_pandas import read_root

In [ ]:
rootpath = '/home/jcordero/CMS/data/data_2017/rereco/SMP_ZG/Files/mumug/VDask/WWTo2L2Nu/output_WWTo2L2Nu_1.root'
#files = TFile(DSample['DYJets'][0].path)
for df in read_root(rootpath,chunksize=10000):
    event = Event(df)

In [ ]:
#PhotonRadiation = None
PhotonRadiation = 'IRS'
OppositeCharge = True
#VetoDY = True
VetoDY = False

Cutting = Cuts(PhotonRadiation = PhotonRadiation, OppositeCharge = OppositeCharge, VetoDY = VetoDY)

In [ ]:
flag = Cutting._DR_l1gm(event)
print('DRl1',np.sum(flag))
#flag = np.logical_and(flag,Cutting._DR_l2gm(event))
flag *= Cutting._DR_l2gm(event)
print('DRl2',np.sum(flag))

if Cutting.MVA is not None and self.MVA: 
    flag *= Cutting._MVA(event)
print('MVA',np.sum(flag))
    
if Cutting.PhotonRadiation is not None:
    if Cutting.PhotonRadiation == 'ISR': 
        flag *= Cutting._ISR(event)
    elif Cutting.PhotonRadiation == 'FSR': 
        flag *= Cutting._ISR(event)
print('PhotonType',np.sum(flag))
        
# Implemented on the main code and not in this STD member 
'''
if self.VetoDY is not None:
    flag *= self._vetoDY(event)
'''

if Cutting.OppositeCharge is not None: 
    if Cutting.OppositeCharge:
        flag *= Cutting._oppositeCharge(event)
    else: 
        flag *= Cutting._sameCharge(event)
print('Charge',np.sum(flag))
        
flag *= Cutting._extremeWeight(event)
print(np.sum(flag))

In [ ]:
np.sum(event.event.l1PhotonDR > 0.7)

In [ ]:
np.sum(event.event.l2PhotonDR > 0.7)

In [ ]:
np.sum((event.event.l1PhotonDR > 0.7)*(event.event.l2PhotonDR > 0.7))

In [ ]:
np.sum(Cutting._oppositeCharge(event))

In [ ]:
for sample in Samples:
    print(sample,DSample[sample].getSF())

In [ ]:
newpath = "/home/jcordero/CMS/data/data_2017/rereco/SMP_ZG/Files/mumug/VDask/"
oldpath = "/home/jcordero/CMS/data/data_2017/rereco/SMP_ZG/Files/mumug/V6_lPhoton/"
oldsmppath = 'WWTo2L2Nu/output_WWTo2L2Nu_v_0.root'
OLD = read_root(oldpath+oldsmppath)['eventWeight']

#files = TFile(DSample['DYJets'][0].path)
weight = []
NEW = pd.DataFrame()
for i in range(1,11):
    newsmppath = 'WWTo2L2Nu/output_WWTo2L2Nu_{}.root'.format(str(i))
    event = Event(read_root(newpath+newsmppath))
    NEW = NEW.append(read_root(newpath+newsmppath))

In [ ]:
plt.hist(OLD)


In [ ]:
plt.hist(NEW['eventWeight'])

In [ ]:
event.value('eventWeight')*DSample['DYJets'].getSF()